In [8]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

asyncio.run(asyncio.sleep(1))

# import warnings
# warnings.filterwarnings('ignore')
# warnings.filterwarnings('error', category=DeprecationWarning)

# import logging
# logger = logging.getLogger('stan')
# logger.addHandler(logging.NullHandler())

In [2]:
import numpy as np 
from Models.BWM import StandardBWM, GaussianBWM
from Graphs import credal_ranking

In [3]:
def CredalRanking(weights, c_name):
    import xarray as xr
    xr.set_options(display_style="html");
    from pyvis.network import Network

    avg_weights = np.mean(weights, axis=0)
    index = np.argsort(-avg_weights)

    sample_no, c_no = weights.shape
    criteria_name = c_name.copy()
    #for i in range(c_no):
    #    criteria_name[i] = criteria_name[i] + ' - ' + str(round(avg_weights[i],3))

    probs=np.empty((c_no, c_no))
    for i in range(c_no):
        for j in range(i, c_no):
            probs[i,j] = round((weights[:,i] >= weights[:,j]).sum() / sample_no,2)
            probs[j,i] = 1 - probs[i,j]

    ## Visualization using pyvis
    net= Network(notebook=True, layout=None, height='800px', width='600px', directed=True, cdn_resources='local')#'in_line')
    for i in range(c_no):
        net.add_node(str(index[i]), size=max(avg_weights[index[i]]*100,10),
            title=criteria_name[index[i]], label=criteria_name[index[i]], x=0, y=i*200)

    for i in range(c_no-1):
        net.add_edge(str(index[i]), str(index[i+1]), label=str(probs[index[i],index[i+1]]))
        for j in range(i+2, c_no):
            if probs[index[i], index[j]] < 1 and probs[index[i], index[j]] > 0.5:
                net.add_edge(str(index[i]), str(index[j]), label=str(probs[index[i],index[j]]))

    net.toggle_physics(False)
    net.set_edge_smooth("curvedCW")
    #net.show_buttons(filter_=[])
    #net.prep_notebook()

    net.save_graph("simple_graph.html")
    from IPython.display import HTML
    display(HTML(filename="simple_graph.html"))

    return net
    


In [4]:
a_b =  np.array([
    [ 3, 4, 6, 1, 5, 2, 9, 7],
    [ 1, 2, 8, 4, 5, 3, 9, 6],
    [ 2, 2, 3, 1, 5, 5, 9, 8],
    [ 2, 1, 8, 2, 9, 3, 8, 8],
    [ 2, 4, 9, 1, 4, 3, 5, 5],
    [ 1, 2, 9, 1, 3, 5, 5, 4],
    ])

a_w =  np.array([
    [ 7, 6, 4, 9, 5, 8, 1, 3],
    [ 9, 8, 2, 5, 4, 5, 1, 3],
    [ 8, 8, 5, 9, 5, 5, 1, 2],
    [ 8, 9, 2, 8, 1, 8, 2, 2],
    [ 8, 6, 1, 9, 6, 7, 4, 4],
    [ 9, 8, 1, 9, 7, 5, 5, 6],
    ])

In [5]:
bwm = StandardBWM.StandardBWM( AB=a_b, AW=a_w)
bwm.sampling()

The used model is:  self._basicModel
Building...



Building: found in cache, done.Messages from stanc:
Warning in '/var/folders/yg/6v5nv1f56wj23g741x_8qpr40000gn/T/httpstan_dfvq5l4n/model_zz3o6ikf.stan', line 34, column 38: Argument .001 suggests there may be parameters that are not unit scale; consider rescaling with a multiplier (see manual section 22.12).
Warning in '/var/folders/yg/6v5nv1f56wj23g741x_8qpr40000gn/T/httpstan_dfvq5l4n/model_zz3o6ikf.stan', line 34, column 33: There is a gamma or inverse-gamma distribution with parameters that are equal to each other and set to values less than 1. This is mathematically acceptable and can make sense in some problems, but typically we see this model used as an attempt to assign a noninformative prior distribution. In fact, priors such as inverse-gamma(.001,.001) can be very strong, as explained by Gelman (2006). Instead we recommend something like a normal(0,1) or student_t(4,0,1), with parameter constrained to be positive.
Warning in '/var/folders/yg/6v5nv1f56wj23g741x_8qpr40000gn/T/h

In [6]:
from IPython.display import HTML,display

display(HTML(filename="simple_graph.html"))

In [7]:
criteria_name = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8']

#credal_ranking.plot(bwm.AggregatedWeightSamples.T, criteria_name)

net = CredalRanking(bwm.AggregatedWeightSamples.T, criteria_name)

In [39]:
net.write_html("network.html", local=True, notebook=True)


In [44]:
from IPython.display import display, HTML

with open("./simple_graph.html", "r", encoding="utf-8") as f:
    contents = f.read()
    #print(contents)
    display(HTML(contents))

#IFrame(src="./simple_graph.html", width=700, height=600)